# 人工知能 csc.t272
## プロジェクト課題
### MNIST dataset をもちいた文字生成

In [4]:
# パッケージのインポート
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm

In [5]:
# model の作成
# MNIST dataset; [28, 28]
class MNIST_NN(nn.Module):
    def __init__(
            self,
            layer_dim=256,
    ):
        super().__init__()
        self.layer1 = nn.Linear(784, layer_dim)
        self.layer2 = nn.Linear(layer_dim, layer_dim)
        self.layer3 = nn.Linear(layer_dim, 10)
        self.act = nn.ReLU()

    def forward(self, input):
        x = input.view(input.size(0), -1) 
        x = self.layer1(x)
        x = self.act(x)
        x = self.layer2(x)
        x = self.act(x)
        x = self.layer3(x)
        return x

In [ ]:
# main
## model やdataset の定義
model = MNIST_NN()

bs = 64
lr = 0.002
epochs = 5

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

train_data = torchvision.datasets.MNIST(root="~/AI/data/train", train=True, download=True, transform=transforms.ToTensor())
test_data = torchvision.datasets.MNIST(root="~/AI/data/test", train=False, download=True, transform=transforms.ToTensor())

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=bs)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=bs)

## training

for epoch in range(epochs):
    model.train()

    total_loss = 0
    train_bar = tqdm(
        enumerate(train_dataloader),
        total=len(train_dataloader),
        desc=f"epoch: {epoch+1} / {epochs}"
    )

    for batch, (img, lbl) in train_bar:
        optimizer.zero_grad()
        pred = model(img)
        loss = criterion(pred, lbl)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        train_bar.set_postfix(loss=f"{loss.item():.4f}")
    
    model.eval()
    with torch.no_grad():
        eval_bar = tqdm(
            enumerate(test_dataloader),
            total=len(test_dataloader),
            desc=f"epoch: {epoch+1} / {epochs}"
        )

        for batch, (img, lbl) in eval_bar:
            pred = model(img)
            loss = criterion(pred, lbl)
            eval_bar.set_postfix(loss=f"{loss:.4f}")

epoch: 1 / 5:   0%|          | 0/938 [00:00<?, ?it/s]


NameError: name 'loss' is not defined

In [8]:
from train import training, evaluation
acc = evaluation(
    model=model,
    dataloader=test_dataloader,
    load=True
)
print(f"accuracy: {acc}")

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'